In [1]:
# Enable Float64 for more stable matrix inversions.
from jax import config
config.update("jax_enable_x64", True)

import numpy as np

import jax.numpy as jnp
from jax import random
from jax import jit

import gpjax as gpx
from kohgpjax.kohkernel import KOHKernel

key = random.PRNGKey(123)

/Users/jamesbriant/miniforge3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [2]:

DATAFIELD = np.loadtxt('data/simple_field.csv', delimiter=',', dtype=np.float32)
DATACOMP = np.loadtxt('data/simple_comp.csv', delimiter=',', dtype=np.float32)

num_obs = DATAFIELD.shape[0]

xf = np.reshape(DATAFIELD[:, 0], (-1, 1))
xc = np.reshape(DATACOMP[:, 0], (-1,1))
tc = np.reshape(DATACOMP[:, 1], (-1,1))
yf = np.reshape(DATAFIELD[:, 1], (-1,1))
yc = np.reshape(DATACOMP[:, 2], (-1,1))



#Standardize full response using mean and std of yc
yc_mean = np.mean(yc)
# yc_std = np.std(yc)
yc_std = np.std(yc, ddof=1) #estimate is now unbiased
x_min = min(xf.min(), xc.min())
x_max = max(xf.max(), xc.max())
t_min = tc.min()
t_max = tc.max()

xf_normalized = (xf - x_min)/(x_max - x_min)
xc_normalized = (xc - x_min)/(x_max - x_min)
# tc_normalized = np.zeros_like(tc)
# for k in range(tc.shape[1]):
#     tc_normalized[:, k] = (tc[:, k] - np.min(tc[:, k]))/(np.max(tc[:, k]) - np.min(tc[:, k]))
tc_normalized = (tc - t_min)/(t_max - t_min)
yc_standardized = (yc - yc_mean)/yc_std
yf_standardized = (yf - yc_mean)/yc_std

theta = 0.5

x = jnp.vstack((xf_normalized, xc_normalized))
t = jnp.vstack((jnp.zeros((xf_normalized.shape[0], tc_normalized.shape[1])) + theta, tc_normalized))
x = jnp.hstack((x, t), dtype=np.float64)
y = jnp.vstack((yf_standardized, yc_standardized), dtype=np.float64)

data = gpx.Dataset(X=x, y=y)

# Create Prior

In [3]:

product_kernel = gpx.kernels.ProductKernel(kernels=[
        gpx.kernels.RBF(
            active_dims=[0],
            lengthscale=jnp.array(1/jnp.sqrt(2*50)),
        ), 
        gpx.kernels.RBF(
            active_dims=[1],
            lengthscale=jnp.array(1/jnp.sqrt(2*7)),
        )
    ])

kernel = KOHKernel(
    num_obs=num_obs,
    k_eta=product_kernel,
    k_delta=gpx.kernels.White(
        active_dims=[0],
        # lengthscale=jnp.array(1/jnp.sqrt(2*2)),
        variance=jnp.array(1/30)
    ), 
    k_epsilon=gpx.kernels.White(
        active_dims=[0],
        variance=jnp.array(1/1000)
    ),
    k_epsilon_eta=gpx.kernels.White(
        active_dims=[0],
        variance=jnp.array(1/10000)
    ),
)
meanf = gpx.mean_functions.Zero()
prior = gpx.Prior(
    mean_function=meanf, 
    kernel=kernel,
    jitter=0.
)

# Build Posterior

In [4]:
likelihood = gpx.likelihoods.Gaussian(
    num_datapoints=data.n,
    obs_stddev=jnp.array(0.0)
)

posterior = prior * likelihood

negative_mll = jit(gpx.objectives.ConjugateMLL(negative=True))
nll = negative_mll(posterior, train_data=data)

print(nll - 0.5*data.n*jnp.log(2*jnp.pi))

-740.7842095061286
